In [2]:
import pandas as pd
import data_handler_new
from sqlalchemy.sql import text
import json

dh = data_handler_new.DataHandlerProduction()
# Database validation metrics
df_metrics = pd.DataFrame(columns={'dossier','nt-basis','all'}, index=['nr_of_docs', 
                                                                       'nr_parsed', 
                                                                       'ratio_parsed',
                                                                       'ratio_w_pico',
                                                                       'ratio_w_ind',
                                                                       'ratio_w_comp',
                                                                       ])

# connect to datahandler
def get_result(query):
    result = dh.query_database(query, retries=5) 

    return result.fethone()

SAVE_PATH = 'C:/Users/stahl-pnordics/OneDrive - SmartStep Consulting GmbH/python/tlv/'

FILENAME = SAVE_PATH + 'links.json'
with open(FILENAME, 'r') as f:
            links = json.load(f)

nr_of_dossiers = len(links)

FILENAME = SAVE_PATH + 'nt_links.json'
with open(FILENAME, 'r') as f:
            links = json.load(f)

nr_of_nt_basis = len(links)

nr_of_all = nr_of_dossiers + nr_of_nt_basis

nr_of_docs = {'dossier': nr_of_dossiers, "nt-basis": nr_of_nt_basis, "all": nr_of_all}

for doc_type in ['dossier','nt-basis','all']:
    if doc_type=='all':
        post = '!=None'   
    else:
        post = '='+doc_type 
    df_metrics.iloc['nr_of_docs', doc_type] = nr_of_docs[doc_type]

    # Nr of hta documents vs in links
    query = text("SELECT count(*) FROM hta_documents WHERE document_type{}".format(doc_type))
    df_metrics.iloc['nr_parsed', doc_type] = get_result(query)
    df_metrics.iloc['ratio_parsed', doc_type] = 100* df_metrics.iloc['nr_parsed', doc_type]/nr_of_docs[doc_type]

    # Nr of hta documents with picos
    query = text("SELECT count(distinct(idhta_document)) FROM pico p JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_pico', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of hta documents with indications
    query = text("SELECT count(distinct(idhta_document)) FROM hta_document_indication hdi JOIN hta_document h ON hdi.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_ind', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of hta documents without company
    query = text("SELECT count(*) FROM hta_document WHERE idcompany is NULL AND document_type='dossier'")
    df_metrics.iloc['ratio_w_comp', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of hta documents with product
    query = text("SELECT count(distinct(idhta_document)) FROM hta_document_product hdp JOIN hta_document h ON hdp.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_prod', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of hta documents with staff
    query = text("SELECT count(distinct(idhta_document)) FROM hta_document_staff hds JOIN hta_document h ON hds.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_staff', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of hta documents with experts
    query = text("SELECT count(distinct(idhta_document)) FROM hta_document_has_expert he hdp JOIN hta_document h ON he.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_expert', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_parsed', doc_type]

    # Nr of picos without analysis
    query = text("SELECT count(distinct(p.id)) FROM pico p JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'")
    nr_pico = get_result(query)
    
    query = text("SELECT count(distinct(idpico)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'")
    df_metrics.iloc['ratio_w_analysis', doc_type] = 100* get_result(query) / nr_pico

    # Nr of CE
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
             WHERE analysis_type='cost-effectiveness'""")
    df_metrics.iloc['nr_w_CE', doc_type] = get_result(query)

    # Nr of CE with ICER company
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
             WHERE analysis_type='cost-effectiveness' AND ICER_company IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_comp_ICER', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CE with ICER agency lower
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'
                WHERE analysis_type='cost-effectiveness' AND ICER_agency_lower IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_agency_ICER_low', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CE with ICER agency higher
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'
                WHERE analysis_type='cost-effectiveness' AND ICER_agency_higher IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_agency_ICER_high', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CE without QALY comp
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier'
                WHERE analysis_type='cost-effectiveness' AND QALY_company IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_comp_QALY', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CE with QALY agency lower
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
                WHERE analysis_type='cost-effectiveness' AND QALY_agency_lower IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_ag_QALY_low', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CE with QALY agency higher
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
                WHERE analysis_type='cost-effectiveness'AND QALY_agency_higher IS NOT NULL""")
    df_metrics.iloc['ratio_CE_w_ag_QALY_high', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_w_CE', doc_type]

    # Nr of CM
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
                JOIN costs c ON a.id=c.idanalysis WHERE analysis_type='cost-minimization'""")
    df_metrics.iloc['nr_CM', doc_type] = get_result(query) 

    # Nr of CM with prices company
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
                JOIN costs c ON a.id=c.idanalysis WHERE analysis_type='cost-minimization' AND c.assessor='company' AND c.drug_cost IS NOT NULL""")
    df_metrics.iloc['ratio_CM_w_comp_costs', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_CM', doc_type]

    # Nr of CM without prices agency
    query = text("""SELECT COUNT(DISTINCT(a.id)) FROM analysis a JOIN pico p ON a.idpico=p.id JOIN hta_document h ON p.idhta_document=h.id AND h.document_type='dossier' 
                JOIN costs c ON a.id=c.idanalysis WHERE analysis_type='cost-minimization' AND c.assessor='agency' AND c.drug_cost IS NOT NULL""")
    df_metrics.iloc['ratio_CM_w_ag_costs', doc_type] = 100* get_result(query) / df_metrics.iloc['nr_CM', doc_type]

# Nr of analysis with trials
#query = text("SELECT COUNT(DISTINCT(idanalysis)) FROM documents_db_5.trial")
#query = text("SELECT count(distinct(*)) FROM analysis")
# Nr of trials without outcome values

# Nr of trials without reference


ModuleNotFoundError: No module named 'sandbox'